In [83]:
from google.colab import drive
from sklearn.inspection import permutation_importance
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from pandas.core.common import random_state
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import joblib

In [84]:
drive.mount('/content/drive')
%cd drive/MyDrive/ML_final
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: 'drive/MyDrive/ML_final'
/content/drive/MyDrive/ML_final


In [85]:
data = pd.concat([train, test])
leave_col = [f for f in data.columns if f.startswith('measurement')
             or f == 'id' or f == 'product_code' or f == 'loading' or f
             == 'failure']
data = data[leave_col]
print(data.shape)

(47345, 22)


In [86]:
data['m3_missing'] = data['measurement_3'].isnull().astype(np.int8)
data['m5_missing'] = data['measurement_5'].isnull().astype(np.int8)

In [87]:
pro_code = [
    'A',
    'B',
    'C',
    'D',
    'E',
    'F',
    'G',
    'H',
    'I',
    ]
fill_col = [f for f in data.columns if f.startswith('measurement') or f
            == 'loading']
for code in pro_code:
    tem_data = data[data.product_code == code].drop(['failure',
            'product_code', 'id', 'm3_missing', 'm5_missing'], axis=1)
    model_knn = KNNImputer(n_neighbors=5)
    tem_data = model_knn.fit_transform(tem_data)
    data.loc[data.product_code == code, fill_col] = tem_data

In [88]:
data_scal = data[fill_col]
scaler = StandardScaler().fit(data_scal)
data_scal = scaler.transform(data_scal)
data[fill_col] = data_scal

In [89]:
train_data = data[data.failure.notnull()]
test_data = data[data.failure.isnull()]

In [90]:
train_y = train_data.failure.astype(int)
train_x = train_data.drop(['failure', 'product_code', 'id'], axis=1)
test_x = test_data.drop(['failure', 'product_code', 'id'], axis=1)

In [91]:
# best_feature = [
#     'measurement_2',
#     'loading',
#     'measurement_17',
#     'm3_missing',
#     'm5_missing',
#     'measurement_1',
#     ]

In [92]:
select_f = [
    'measurement_2',
    'loading',
    'measurement_17',
    'measurement_1',
    'm3_missing',
    'm5_missing'
    ]
train_x_s = train_x[select_f]
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
model_list = []
for (indfk, (tra, val)) in enumerate(kf.split(train_x_s, train_y)):
    t_x = train_x_s.iloc[tra]
    t_y = train_y.iloc[tra]
    v_x = train_x_s.iloc[val]
    v_y = train_y.iloc[val]
    model = LogisticRegression(max_iter=1000, C=0.0001, penalty='l2',
                               solver='newton-cg')
    model.fit(t_x, t_y)
    model_list.append(model)
    pred = model.predict_proba(v_x)
    pred = pred[:, 1]

In [93]:
test_x_s = test_x[select_f]
model = LogisticRegression(max_iter=1000, C=0.01, penalty='l2',
                           solver='newton-cg')
model.fit(train_x_s, train_y)


LogisticRegression(C=0.01, max_iter=1000, solver='newton-cg')

In [94]:
joblib.dump(model, 'final_model')

['final_model']